### *PhotoDissociation Region Toolbox Notebooks*
-------------------------------------------------------------

# Example 7: Adding Custom Models

This example shows how you can add a new model to a `ModelSet`. Here we show adding a ratio, but you can add an intensity in a similar fashion.

In [1]:
from pdrtpy.modelset import ModelSet
from pdrtpy import pdrutils as utils
from copy import deepcopy

### Instantiate a `ModelSet` and see what's in it.

In [2]:
ms = ModelSet("wk2020",z=1)
ms.supported_ratios.show_in_notebook()

idx,title,ratio label
0,[O I] 63 $\mu$m / [C II] 158 $\mu$m,OI_63/CII_158
1,([O I] 63 $\mu$m + [C II] 158 $\mu$m) / I$_{FIR}$,OI_63+CII_158/FIR
2,([O I] 145 $\mu$m + [C II] 158 $\mu$m) / I$_{FIR}$,OI_145+CII_158/FIR
3,[O I] 145 $\mu$m / [O I] 63 $\mu$m,OI_145/OI_63
4,[C I] 370 $\mu$m / [C I] 609 $\mu$m,CI_370/CI_609
5,[C II] 158 $\mu$m / [C I] 609 $\mu$m,CII_158/CI_609
6,[C II] 158 $\mu$m / [O I] 145 $\mu$m,CII_158/OI_145
7,[C II] 158 $\mu$m / I$_{FIR}$,CII_158/FIR
8,[C II] 158 $\mu$m / CO(J=1-0),CII_158/CO_10
9,[C II] 158 $\mu$m / CO(J=2-1),CII_158/CO_21


## 1. Adding a new model to an existing ModelSet

### Let's say you want to add C[II] 158$\mu$m/CO(4-3), which doesn't exist in this ModelSet


In [3]:
try:
    ms.get_model("CII_158/CO_43")
except KeyError:
    print("proof that this model is not already in the ModelSet")

proof that this model is not already in the ModelSet


### You can make it arithmetically from the C[II] 158$\mu$m/CO(2-1) and CO(4-3)/CO(2-1) models.
`Measurements` obey arithmetic operators and correctly propogate units and errors

In [4]:
ciico21 = ms.get_model("CII_158/CO_21")
co43co21 = ms.get_model("CO_43/CO_21")
ciico43 = ciico21/co43co21

### The header of the new `Measurement` will be empty.
You can create one by copying an existing header and modifying it. Initially the new header has minimal information.

In [5]:
ciico43.header

OrderedDict([('BUNIT', ''), ('BMAJ', None), ('BMIN', None), ('BPA', None)])

In [6]:
ciico21.header

SIMPLE  =                    T /                                                
BITPIX  =                  -32 /                                                
NAXIS   =                    2                                                  
NAXIS1  =                   49 /                                                
NAXIS2  =                   57 /                                                
COMMENT  FITS (Flexible Image Transport System) format is defined in 'Astronomy 
COMMENT  and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H  
COMMENT  nemo::fitsio.c $Id$                                                    
DATASUM = '0000000000000000'                                                    
CHECKSUM= '0000000000000000'                                                    
TITLE   = '[CII] 158 micron/CO (J=2-1)'                                         
BUNIT   = ' '                                                                   
ORIGIN  = 'UMD'             

#### Copy the [CII] 158 $\mu$m / CO (J=2-1) header and update it.


In [7]:
ciico43.header = deepcopy(ciico21.header)
ciico43.header["TITLE"] = '[CII] 158 micron/CO (J=4-3)' 
ciico43.header["DATAMAX"] = ciico43.data.max()
ciico43.header["DATAMIN"] = ciico43.data.min()
ciico43.header["HISTORY"] = "Computed arithmetically from (CII_158/CO_21)/(CO_43/CO_21)"
ciico43.header["DATE"] = utils.now() #the current time and date
ciico43.header

SIMPLE  =                    T /                                                
BITPIX  =                  -32 /                                                
NAXIS   =                    2                                                  
NAXIS1  =                   49 /                                                
NAXIS2  =                   57 /                                                
COMMENT  FITS (Flexible Image Transport System) format is defined in 'Astronomy 
COMMENT  and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H  
COMMENT  nemo::fitsio.c $Id$                                                    
DATASUM = '0000000000000000'                                                    
CHECKSUM= '0000000000000000'                                                    
TITLE   = '[CII] 158 micron/CO (J=4-3)'                                         
BUNIT   = ' '                                                                   
ORIGIN  = 'UMD'             

### Now add the model to the `ModelSet`
The `identifier` should follow the convention already used for the individual spectral lines. 
The `title` can contain LaTeX characters.

In [8]:
ms.add_model("CII_158/CO_43",ciico43,title="[C II] 158 $\mu$m / CO(J=4-3)")

Adding user model CII_158/CO_43


### The added model is now listed in available ratios

In [9]:
ms.supported_ratios.show_in_notebook()

idx,title,ratio label
0,[O I] 63 $\mu$m / [C II] 158 $\mu$m,OI_63/CII_158
1,([O I] 63 $\mu$m + [C II] 158 $\mu$m) / I$_{FIR}$,OI_63+CII_158/FIR
2,([O I] 145 $\mu$m + [C II] 158 $\mu$m) / I$_{FIR}$,OI_145+CII_158/FIR
3,[O I] 145 $\mu$m / [O I] 63 $\mu$m,OI_145/OI_63
4,[C I] 370 $\mu$m / [C I] 609 $\mu$m,CI_370/CI_609
5,[C II] 158 $\mu$m / [C I] 609 $\mu$m,CII_158/CI_609
6,[C II] 158 $\mu$m / [O I] 145 $\mu$m,CII_158/OI_145
7,[C II] 158 $\mu$m / I$_{FIR}$,CII_158/FIR
8,[C II] 158 $\mu$m / CO(J=1-0),CII_158/CO_10
9,[C II] 158 $\mu$m / CO(J=2-1),CII_158/CO_21


### Write out the model so you can use it later

In [10]:
ciico43.write("CII_CO43.fits",overwrite=True)

### You can retrieve it just like you do with other models

In [11]:
ms.get_model("CII_158/CO_43")

[[3.4678601e+02 2.0818153e+02 1.0749241e+02 ... 1.7362282e-07
  2.5683008e-07 3.9756594e-07]
 [4.9616333e+02 2.9232959e+02 1.4560727e+02 ... 2.4507924e-07
  2.2378157e-07 2.4590031e-07]
 [8.0963776e+02 4.6750064e+02 2.2254070e+02 ... 4.0613224e-07
  2.5959119e-07 1.8995189e-07]
 ...
 [4.3223555e+06 3.8011560e+06 3.1338668e+06 ... 2.0164965e+01
  1.9100477e+01 1.8327578e+01]
 [5.7499820e+06 5.0302995e+06 4.1132158e+06 ... 2.1112997e+01
  2.0044981e+01 1.9266415e+01]
 [6.9362080e+06 6.0495800e+06 4.9228095e+06 ... 2.1759483e+01
  2.0691019e+01 1.9882364e+01]] +/- None 

### Check what models have been added

In [12]:
ms.user_added_models

['CII_158/CO_43']

#### Once added, you can use [C II] 158 $\mu$m / CO(J=4-3) in $(n,G_0)$ fitting as you would with any other model

-----------------------

## 2. Replacing a model that is already in the `ModelSet`
The procedure is similar, but you have to explicitly overwrite the existing model.
In this example we take CO(J=2-1) from the ModelSet, scale it by 1.5 and write it back.

In [13]:
scaledciico21 =  ciico21 * 1.5
scaledciico21.header = deepcopy(ciico21.header)
scaledciico21.header["DATAMAX"] = ciico21.data.max()
scaledciico21.header["DATAMIN"] = ciico21.data.min()
scaledciico21.header["HISTORY"] = "Rescaled (CII_158/CO_21) * 1.5"
scaledciico21.header["DATE"] = utils.now()

If you don't specifiy `overwrite=True` an exception will be raised.

In [14]:
try:
    ms.add_model("CII_158/CO_21",scaledciico21,title="[C II] 158 $\mu$m / CO(J=2-1)")
except Exception as exc:
    print(exc)

CII_158/CO_21 is already in the wk2020 ModelSet. If you wish to overwrite it, use overwrite=True


In [15]:
ms.add_model("CII_158/CO_21",scaledciico21,title="[C II] 158 $\mu$m / CO(J=2-1)",overwrite=True)
ms.get_model("CII_158/CO_21").header["HISTORY"]


Adding user model CII_158/CO_21



PhotoDissociation Region Toolbox
http://dustem.astro.umd.edu
http://pdrtpy.readthedocs.io
Rescaled (CII_158/CO_21) * 1.5

In [16]:
ms.supported_ratios

title,ratio label
str51,str22
[O I] 63 $\mu$m / [C II] 158 $\mu$m,OI_63/CII_158
([O I] 63 $\mu$m + [C II] 158 $\mu$m) / I$_{FIR}$,OI_63+CII_158/FIR
([O I] 145 $\mu$m + [C II] 158 $\mu$m) / I$_{FIR}$,OI_145+CII_158/FIR
[O I] 145 $\mu$m / [O I] 63 $\mu$m,OI_145/OI_63
[C I] 370 $\mu$m / [C I] 609 $\mu$m,CI_370/CI_609
[C II] 158 $\mu$m / [C I] 609 $\mu$m,CII_158/CI_609
[C II] 158 $\mu$m / [O I] 145 $\mu$m,CII_158/OI_145
...,...
[Fe II] 17.94 $\mu$m/[Fe II] 5.34 $\mu$m,FEII_17.94/FEII_5.34


### If you want to reset everything, just reinstantiate the `ModelSet`

In [17]:
ms = ModelSet("wk2020",z=1)

### If you wanted to add it later, you can give the path to the file in `add_model`

In [18]:
ms.add_model("CII_158/CO_43","CII_CO43.fits",title="[C II] 158 $\mu$m / CO(J=4-3)")

Adding user model CII_158/CO_43


In [19]:
ms.supported_ratios

title,ratio label
str51,str22
[O I] 63 $\mu$m / [C II] 158 $\mu$m,OI_63/CII_158
([O I] 63 $\mu$m + [C II] 158 $\mu$m) / I$_{FIR}$,OI_63+CII_158/FIR
([O I] 145 $\mu$m + [C II] 158 $\mu$m) / I$_{FIR}$,OI_145+CII_158/FIR
[O I] 145 $\mu$m / [O I] 63 $\mu$m,OI_145/OI_63
[C I] 370 $\mu$m / [C I] 609 $\mu$m,CI_370/CI_609
[C II] 158 $\mu$m / [C I] 609 $\mu$m,CII_158/CI_609
[C II] 158 $\mu$m / [O I] 145 $\mu$m,CII_158/OI_145
...,...
[Fe II] 5.67 $\mu$m/[Fe II] 5.34 $\mu$m,FEII_5.67/FEII_5.34
